In [1]:
import pandas as pd
import numpy as np
from music21 import converter, midi, interval, pitch
from mido import MidiFile
import miditoolkit
import os
from os import walk
import json
from tokenizing_functions import extract_events, get_file_and_dirnames
#from helper_functions import get_file_and_dirnames
#from analysis_functions import analyse_data_folder
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

PATH_TRANSPOSED = "../0_data/4_preprocessed_sets"
PATH_VOCAB = "../0_data/5_vocabs"
PATH_WORD_DATA = "../0_data/6_word_data"

for path in [PATH_VOCAB, PATH_WORD_DATA]:
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:
vocab_configs = {
    "a1" : {
        "pitch_start": 0,
        "pitch_end": 127,
        "duration_steps": 64,
        "triole_tokens": False,
        "folder_name": "a)_4_4_metric_120_bpm"
    },
    "a2" : {
        "pitch_start": 0,
        "pitch_end": 127,
        "duration_steps": 64,
        "triole_tokens": True,
        "folder_name": "a)_4_4_metric_120_bpm"
    },
    "a3" : {
        "pitch_start": 0,
        "pitch_end": 127,
        "duration_steps": 32,
        "triole_tokens": False,
        "folder_name": "a)_4_4_metric_120_bpm"
    },
    "b" : {
        "pitch_start": 0,
        "pitch_end": 127,
        "duration_steps": 64,
        "triole_tokens": False,
        "folder_name": "b)_transposed_key"
    },
    "c" : {
        "pitch_start": 60,
        "pitch_end": 95,
        "duration_steps": 64,
        "triole_tokens": False,
        "folder_name": "c)_transposed_octave"
    },
    "d" : {
        "pitch_start": 60,
        "pitch_end": 95,
        "duration_steps": 32,
        "triole_tokens": True,
        "folder_name": "d)_transposed_key_and_octave"
    }
}

In [3]:
POSITION_STEPS = 16
TICKS_PER_BEAT = 1024
TRIOLE_POS_1 = (TICKS_PER_BEAT/12).__round__()
TRIOLE_POS_2 = (TICKS_PER_BEAT/6).__round__()
TICKS_PER_MIN_DURATION = TICKS_PER_BEAT*4/32

for key in vocab_configs:

    duration_steps = vocab_configs[key]["duration_steps"]
    pitch_start = vocab_configs[key]["pitch_start"]
    pitch_end = vocab_configs[key]["pitch_end"]
    pitch_range = pitch_end - pitch_start + 1
    triole_tokens = vocab_configs[key]["triole_tokens"]

    # 1 Bar token
    token2word = {0: "Bar_None"}
    # 36 or 128 Note-On tokens
    for i in range(1, pitch_range+1):
        token2word[i] = f"Note-On_{i+pitch_start-1}"
    # 32 or 64 Note-Duration tokens
    for i in range(pitch_range+1, pitch_range+duration_steps+1):
        token2word[i] = f"Note-Duration_{i-pitch_range}"
    # 1 duration triole token if triole_tokens is True
    if triole_tokens:
        token2word[pitch_range+duration_steps+1] = "Note-Duration_triole"
        start_position_tokens = pitch_range+duration_steps+2
    else:
        start_position_tokens = pitch_range+duration_steps+1
    # 16 Position tokens
    for i in range(start_position_tokens, start_position_tokens+POSITION_STEPS):
        token2word[i] = f"Position_{i-start_position_tokens+1}/{POSITION_STEPS}"
    # 2 Position triole tokens if triole_tokens is True
    if triole_tokens:
        token2word[start_position_tokens+POSITION_STEPS] = "Position-Triole_1"
        token2word[start_position_tokens+POSITION_STEPS+1] = "Position-Triole_2"

    word2token = {v: k for k, v in token2word.items()}
    
    with open(f"{PATH_VOCAB}/vocab_{key}.json", "w") as fp:
        json.dump(word2token, fp)

In [4]:
for key in vocab_configs:

    # get files
    folder_name = vocab_configs[key]["folder_name"]
    dir = f"{PATH_TRANSPOSED}/{folder_name}/17_POP909-Dataset-master"
    files,_ = get_file_and_dirnames(dir)
    files.sort()

    # process midi files into word sequences
    word_data = {}
    for file in tqdm(files):
        path = f"{dir}/{file}"
        events =extract_events(path, vocab_configs[key]["duration_steps"], vocab_configs[key]["triole_tokens"])
        words = [f"{e['name']}_{e['value']}" for e in events]
        word_data[file] = words
    print(key, ": ", len(word_data))

    with open(f"{PATH_WORD_DATA}/{key}_data.json", "w") as fp:
        json.dump(word_data, fp)

100%|██████████| 902/902 [00:19<00:00, 45.91it/s]


a1 :  902


100%|██████████| 902/902 [00:23<00:00, 37.74it/s]


a2 :  902


100%|██████████| 902/902 [00:19<00:00, 45.82it/s]


...

b :  845


100%|██████████| 858/858 [00:18<00:00, 45.84it/s]


c :  858


100%|██████████| 803/803 [00:20<00:00, 39.56it/s]


d :  803


In [13]:
def token_to_event(tokens, token2word):
    events = []
    for token in tokens:
        event_name, event_value = token2word.get(token).split('_')
        events.append({
            "name": event_name,
            "time": None,
            "value": event_value,
            "text": None
        })
    return events

def get_position_triole(flags, position, triole_position):
    if triole_position == 0:
        st = flags[position]
    elif triole_position == 1:
        st = flags[position] + TRIOLE_POS_1
    elif triole_position == 2:
        st = flags[position] + TRIOLE_POS_2
    return st

DURATION_BINS = np.arange(TICKS_PER_MIN_DURATION, (TICKS_PER_MIN_DURATION*duration_steps)+1, TICKS_PER_MIN_DURATION, dtype=int)

def write_midi(tokens, token2word, output_path):
    events = token_to_event(tokens, token2word)
    # get downbeat and note (no time)
    incorrect_notes = 0
    temp_notes = []
    for i in range(len(events)-3):
        if events[i]["name"] == 'Bar' and i > 0:
            temp_notes.append('Bar')
        elif events[i]["name"] == 'Position':
            # get position bin 
            position = int(events[i]["value"].split('/')[0]) - 1
            # get triole position
            if events[i+1]["name"] == 'Position-Triole':
                triole_position = int(events[i+1]["value"])
                n = 1
            else:
                triole_position = 0
                n = 0
            if events[i+n+1]["name"] == 'Note-On' and \
            events[i+n+2]["name"] == 'Note-Duration' and events[i+n+2]["value"] != 'triole':
                # pitch
                pitch = int(events[i+n+1]["value"])
            else:
                incorrect_notes += 1
                continue
            # duration
            if events[i+n+3]["name"] == 'Note-Duration' and events[i+n+3]["value"] == 'triole':
                index = int(events[i+n+2]["value"])-1
                duration = int(DURATION_BINS[index] / 3)
            else:
                index = int(events[i+n+2]["value"])-1
                duration = DURATION_BINS[index]
            # adding
            temp_notes.append([position, triole_position ,pitch, duration])
    # get specific time for notes
    ticks_per_bar = TICKS_PER_BEAT * 4 # assume 4/4
    notes = []
    current_bar = 0
    for note in temp_notes:
        if note == 'Bar':
            current_bar += 1
        else:
            position, triole_position, pitch, duration = note
            # position (start time)
            current_bar_st = current_bar * ticks_per_bar
            current_bar_et = (current_bar + 1) * ticks_per_bar
            flags = np.linspace(current_bar_st, current_bar_et, POSITION_STEPS, endpoint=False, dtype=int)
            st = get_position_triole(flags, position, triole_position)
            # duration (end time)
            et = st + duration
            notes.append(miditoolkit.Note(100, pitch, st, et))
    # write to midi
    midi = miditoolkit.midi.parser.MidiFile()
    midi.ticks_per_beat = TICKS_PER_BEAT
    # write instrument
    inst = miditoolkit.midi.containers.Instrument(0, is_drum=False)
    inst.notes = notes
    midi.instruments.append(inst)
        
    # write
    midi.dump(output_path)
    print("midi saved in {}".format(output_path))
    print("Number of incorrect notes: {}".format(incorrect_notes))
    return incorrect_notes

In [14]:
PATH_TEST = "../0_data/99_test"

# when paths not exist, create directories
if not os.path.exists(PATH_TEST):
    os.makedirs(PATH_TEST)

In [15]:
write_midi(tokens, token2word, f"{PATH_TEST}/test.mid")

midi saved in ../0_data/99_test/test.mid
Number of incorrect notes: 0


In [16]:
test_sequences = [[  0, 103, 118,   8, 103, 102,  13,  48, 115,   5,  40, 114, 119,  18,
           8,  38, 109,  15,  52,  84, 112,  20,  44,  40, 111,  12,   8,  38,
         108, 119, 118, 114,  17, 111,  12, 119,  15,  40,  20,  38,  29,  38,
         117,  12,  78, 101, 107,  12,  29,  12,  68,  17,  40,  52, 103,  15,
          44,  13,  38,  22,  46,  15, 118, 104, 119,  11, 104,  25,  40,  77,
          65,  10,  40,  46,   8, 119,  60, 113,  19,  29,  38, 101, 117,  17,
          23,  25,  59,  52,  12, 100,  95,  10,  38, 108,  13, 115,  44, 101,
         113,  17],
        [  0,  94,  44, 101, 111, 119,  15,  52,  65,  38, 103,  22,  17, 101,
         116,  42, 113, 116,  13,  38, 114,  44, 115,  44, 101,  41,  13,  25,
         102, 116,  25,  42, 112, 119,   5,  44,  12, 102,  15,  29, 109,  10,
          10,  10, 105,  15, 112, 119,  26,  13, 107,  12,  59,  48, 101,  44,
         115,  64,  38, 110,  12,  19,  32,   0,  46,   2,   8,  19,  40, 111,
          92,  26, 104,  17, 103, 106,   8,  11,  38,  69, 112, 118,  10,  29,
           6,  56,   8, 112, 106,  15,  19,  52, 104, 119,  42,   0,  55, 104,
         119,  18],
        [  0, 103,  24,  38, 112, 107,  92,  44,   5,  40,   8,  17,  59, 101,
         108,  17,   0,   7,  48, 109, 119,  16, 103, 105,  41,  40,   0,  60,
          15,  38, 106,   5, 116,  17,  40, 107,  17,  44, 110, 108, 109,  15,
          44, 101, 117,  13,  44,  61,  42, 108,  77,  13,   0, 102,  44, 101,
          77,  38, 111,   5,  40, 110,  16,  52, 101, 107,  12,  48,  38,  77,
         109,  18,  52, 104,  84,  26, 111, 112, 118,  25,  19, 113,  20,  38,
           0,   8,  13,  15, 110,  10,  48,  65, 111, 119, 114, 107, 119,  15,
          13,  76]]

write_midi(test_sequences[0], token2word, f"{PATH_TEST}/seq_0.mid")
write_midi(test_sequences[1], token2word, f"{PATH_TEST}/seq_1.mid")
write_midi(test_sequences[2], token2word, f"{PATH_TEST}/seq_2.mid")

midi saved in ../0_data/99_test/seq_0.mid
Number of incorrect notes: 13
midi saved in ../0_data/99_test/seq_1.mid
Number of incorrect notes: 20
midi saved in ../0_data/99_test/seq_2.mid
Number of incorrect notes: 17
